# DATA PREPARATION

To make sure the code will run correctly, please change the file path with your s3 bucket directory in AWS or your local directory

The basic data preparation and loading structure is:
1. Import the files from s3 bucket as data frames
2. Join the data frames with common key 
3. Perform data wrangling (drop unnecessary columns, create new column)
4. Build one data frame with all the informations we will drive insights from
5. Covert this data frame to table for furthur queries

## Import Data

In [1]:
#import airlines data as data frame
airline_df = spark.read.option("inferSchema", "true").option("header", "true").csv("s3://aws-joyce-isom671/finalproject/airlines.csv")
airline_df.take(5)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
13,application_1605922107194_0015,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(IATA_CODE='UA', AIRLINE_NAME='United Air Lines Inc.'), Row(IATA_CODE='AA', AIRLINE_NAME='American Airlines Inc.'), Row(IATA_CODE='US', AIRLINE_NAME='US Airways Inc.'), Row(IATA_CODE='F9', AIRLINE_NAME='Frontier Airlines Inc.'), Row(IATA_CODE='B6', AIRLINE_NAME='JetBlue Airways')]

* Originally in the datasets, there is only one airport file which contains the airport code, name, city, state, country, latitude and longtitude. But since later on in the flights data, there are two columns "ORIGIN_AIRPORT" and "DESTINATION_AIRPORT" that both refer to the airport code in airport data. It would be challening to join the airport data with flights data with the two columns share the common key, so I just copied the aiport file, change the column names correspondingly. One is for origin airport, another one is for desination airport.

In [2]:
#import airport_origin data
#data frame
airport_origin_df = spark.read.option("inferSchema", "true").option("header", "true").csv("s3://aws-joyce-isom671/finalproject/airports-origin.csv")

airport_origin_df.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(IATA_CODE_ORIGIN='ABE', AIRPORT_ORIGIN_NAME='Lehigh Valley International Airport', ORIGIN_CITY='Allentown', ORIGIN_STATE='PA', ORIGIN_COUNTRY='USA', ORIGIN_LATITUDE=40.65236, ORIGIN_LONGITUDE=-75.4404)]

In [3]:
#import airport-dest data
#data frame
airport_dest_df = spark.read.option("inferSchema", "true").option("header", "true").csv("s3://aws-joyce-isom671/finalproject/airports_dest.csv")
airport_dest_df.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(IATA_CODE_DESTINATION='ABE', AIRPORT_DESTINATION_NAME='Lehigh Valley International Airport', DESTINATION_CITY='Allentown', DESTINATION_STATE='PA', DESTINATION_COUNTRY='USA', DESTINATION_LATITUDE=40.65236, DESTINATION_LONGITUDE=-75.4404)]

* In both the airport_origin and airport_destination data frame, columns "LATITUDE" and "LONGTITUDE" are not necessary because the city and state location is already shown in data frame. Also, "COUNTRY" is not necessary since this whole dataset is just about flights in 2015 in Unites States. We will create a new data frame without these 3 columns.

In [4]:
airport_origin_clean_df = airport_origin_df.select("IATA_CODE_ORIGIN","AIRPORT_ORIGIN_NAME","ORIGIN_CITY","ORIGIN_STATE")
airport_origin_clean_df.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(IATA_CODE_ORIGIN='ABE', AIRPORT_ORIGIN_NAME='Lehigh Valley International Airport', ORIGIN_CITY='Allentown', ORIGIN_STATE='PA')]

In [5]:
airport_dest_clean_df = airport_dest_df.select("IATA_CODE_DESTINATION","AIRPORT_DESTINATION_NAME","DESTINATION_CITY","DESTINATION_STATE")
airport_dest_clean_df.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(IATA_CODE_DESTINATION='ABE', AIRPORT_DESTINATION_NAME='Lehigh Valley International Airport', DESTINATION_CITY='Allentown', DESTINATION_STATE='PA')]

## Define Data Structure

In [6]:
#import flights data
#data frame
flight_df = spark.read.option("inferSchema", "true").option("header", "true").csv("s3://aws-joyce-isom671/finalproject/flights.csv")
flight_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [7]:
#Look at the first row in flight_df dataframe
flight_df.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(YEAR=2015, MONTH=1, DAY=1, DAY_OF_WEEK=4, AIRLINE='AS', FLIGHT_NUMBER=98, TAIL_NUMBER='N407AS', ORIGIN_AIRPORT='ANC', DESTINATION_AIRPORT='SEA', SCHEDULED_DEPARTURE=5, DEPARTURE_TIME=2354, DEPARTURE_DELAY=-11, TAXI_OUT=21, WHEELS_OFF=15, SCHEDULED_TIME=205, ELAPSED_TIME=194, AIR_TIME=169, DISTANCE=1448, WHEELS_ON=404, TAXI_IN=4, SCHEDULED_ARRIVAL=430, ARRIVAL_TIME=408, ARRIVAL_DELAY=-22, DIVERTED=0, CANCELLED=0, CANCELLATION_REASON=None, AIR_SYSTEM_DELAY=None, SECURITY_DELAY=None, AIRLINE_DELAY=None, LATE_AIRCRAFT_DELAY=None, WEATHER_DELAY=None)]

* Here we perform left join on the data frame level

In [8]:
#Merge flights data with airline data
first_df = flight_df.join(airline_df, flight_df.AIRLINE == airline_df.IATA_CODE,how='left')
first_df.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(YEAR=2015, MONTH=1, DAY=1, DAY_OF_WEEK=4, AIRLINE='AS', FLIGHT_NUMBER=98, TAIL_NUMBER='N407AS', ORIGIN_AIRPORT='ANC', DESTINATION_AIRPORT='SEA', SCHEDULED_DEPARTURE=5, DEPARTURE_TIME=2354, DEPARTURE_DELAY=-11, TAXI_OUT=21, WHEELS_OFF=15, SCHEDULED_TIME=205, ELAPSED_TIME=194, AIR_TIME=169, DISTANCE=1448, WHEELS_ON=404, TAXI_IN=4, SCHEDULED_ARRIVAL=430, ARRIVAL_TIME=408, ARRIVAL_DELAY=-22, DIVERTED=0, CANCELLED=0, CANCELLATION_REASON=None, AIR_SYSTEM_DELAY=None, SECURITY_DELAY=None, AIRLINE_DELAY=None, LATE_AIRCRAFT_DELAY=None, WEATHER_DELAY=None, IATA_CODE='AS', AIRLINE_NAME='Alaska Airlines Inc.')]

In [9]:
#merge with the airports-origin data frame --> a whole big data frame
second_df = first_df.join(airport_origin_clean_df, first_df.ORIGIN_AIRPORT == airport_origin_clean_df.IATA_CODE_ORIGIN,how='left')
second_df.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(YEAR=2015, MONTH=1, DAY=1, DAY_OF_WEEK=4, AIRLINE='AS', FLIGHT_NUMBER=98, TAIL_NUMBER='N407AS', ORIGIN_AIRPORT='ANC', DESTINATION_AIRPORT='SEA', SCHEDULED_DEPARTURE=5, DEPARTURE_TIME=2354, DEPARTURE_DELAY=-11, TAXI_OUT=21, WHEELS_OFF=15, SCHEDULED_TIME=205, ELAPSED_TIME=194, AIR_TIME=169, DISTANCE=1448, WHEELS_ON=404, TAXI_IN=4, SCHEDULED_ARRIVAL=430, ARRIVAL_TIME=408, ARRIVAL_DELAY=-22, DIVERTED=0, CANCELLED=0, CANCELLATION_REASON=None, AIR_SYSTEM_DELAY=None, SECURITY_DELAY=None, AIRLINE_DELAY=None, LATE_AIRCRAFT_DELAY=None, WEATHER_DELAY=None, IATA_CODE='AS', AIRLINE_NAME='Alaska Airlines Inc.', IATA_CODE_ORIGIN='ANC', AIRPORT_ORIGIN_NAME='Ted Stevens Anchorage International Airport', ORIGIN_CITY='Anchorage', ORIGIN_STATE='AK')]

In [10]:
whole_df = second_df.join(airport_dest_clean_df, second_df.DESTINATION_AIRPORT == airport_dest_clean_df.IATA_CODE_DESTINATION,how='left')
whole_df.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(YEAR=2015, MONTH=1, DAY=1, DAY_OF_WEEK=4, AIRLINE='AS', FLIGHT_NUMBER=98, TAIL_NUMBER='N407AS', ORIGIN_AIRPORT='ANC', DESTINATION_AIRPORT='SEA', SCHEDULED_DEPARTURE=5, DEPARTURE_TIME=2354, DEPARTURE_DELAY=-11, TAXI_OUT=21, WHEELS_OFF=15, SCHEDULED_TIME=205, ELAPSED_TIME=194, AIR_TIME=169, DISTANCE=1448, WHEELS_ON=404, TAXI_IN=4, SCHEDULED_ARRIVAL=430, ARRIVAL_TIME=408, ARRIVAL_DELAY=-22, DIVERTED=0, CANCELLED=0, CANCELLATION_REASON=None, AIR_SYSTEM_DELAY=None, SECURITY_DELAY=None, AIRLINE_DELAY=None, LATE_AIRCRAFT_DELAY=None, WEATHER_DELAY=None, IATA_CODE='AS', AIRLINE_NAME='Alaska Airlines Inc.', IATA_CODE_ORIGIN='ANC', AIRPORT_ORIGIN_NAME='Ted Stevens Anchorage International Airport', ORIGIN_CITY='Anchorage', ORIGIN_STATE='AK', IATA_CODE_DESTINATION='SEA', AIRPORT_DESTINATION_NAME='Seattle-Tacoma International Airport', DESTINATION_CITY='Seattle', DESTINATION_STATE='WA')]

* As you can see from the above data frame result, since we did left join based on the airport code, we will have 4 repeated columns, 2 each for origin and destination. "ORIGIN_AIRPORT" = "IATA_CODE_ORIGIN", "DESTINATION_AIRPORT" = "IATA_CODE_DESTINATION". In this case, we will only keeo "ORIGIN_AIRPORT" and "DESTINATION_AIRPORT"

In [11]:
#drop the repetitive columns
whole_df = whole_df.drop('IATA_CODE_ORIGIN','IATA_CODE_DESTINATION')
whole_df.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(YEAR=2015, MONTH=1, DAY=1, DAY_OF_WEEK=4, AIRLINE='AS', FLIGHT_NUMBER=98, TAIL_NUMBER='N407AS', ORIGIN_AIRPORT='ANC', DESTINATION_AIRPORT='SEA', SCHEDULED_DEPARTURE=5, DEPARTURE_TIME=2354, DEPARTURE_DELAY=-11, TAXI_OUT=21, WHEELS_OFF=15, SCHEDULED_TIME=205, ELAPSED_TIME=194, AIR_TIME=169, DISTANCE=1448, WHEELS_ON=404, TAXI_IN=4, SCHEDULED_ARRIVAL=430, ARRIVAL_TIME=408, ARRIVAL_DELAY=-22, DIVERTED=0, CANCELLED=0, CANCELLATION_REASON=None, AIR_SYSTEM_DELAY=None, SECURITY_DELAY=None, AIRLINE_DELAY=None, LATE_AIRCRAFT_DELAY=None, WEATHER_DELAY=None, IATA_CODE='AS', AIRLINE_NAME='Alaska Airlines Inc.', AIRPORT_ORIGIN_NAME='Ted Stevens Anchorage International Airport', ORIGIN_CITY='Anchorage', ORIGIN_STATE='AK', AIRPORT_DESTINATION_NAME='Seattle-Tacoma International Airport', DESTINATION_CITY='Seattle', DESTINATION_STATE='WA')]

* Create a new column where it calculates the different between arrival delay and departure delay
* Negative value means the arrival delay is less than departure delay, which means the airlines catch up with time between departure and arrival
* Positive value means the arrival delay is bigger than departure delay, which means the airlines was more delayed than it originally was, failed to catch up the delay from origin.

In [12]:
#create a new column
import pyspark.sql.functions as F
whole_df1 = whole_df.withColumn("DELAY_DIFF",F.col("ARRIVAL_DELAY")-F.col("DEPARTURE_DELAY"))
whole_df1.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(YEAR=2015, MONTH=1, DAY=1, DAY_OF_WEEK=4, AIRLINE='AS', FLIGHT_NUMBER=98, TAIL_NUMBER='N407AS', ORIGIN_AIRPORT='ANC', DESTINATION_AIRPORT='SEA', SCHEDULED_DEPARTURE=5, DEPARTURE_TIME=2354, DEPARTURE_DELAY=-11, TAXI_OUT=21, WHEELS_OFF=15, SCHEDULED_TIME=205, ELAPSED_TIME=194, AIR_TIME=169, DISTANCE=1448, WHEELS_ON=404, TAXI_IN=4, SCHEDULED_ARRIVAL=430, ARRIVAL_TIME=408, ARRIVAL_DELAY=-22, DIVERTED=0, CANCELLED=0, CANCELLATION_REASON=None, AIR_SYSTEM_DELAY=None, SECURITY_DELAY=None, AIRLINE_DELAY=None, LATE_AIRCRAFT_DELAY=None, WEATHER_DELAY=None, IATA_CODE='AS', AIRLINE_NAME='Alaska Airlines Inc.', AIRPORT_ORIGIN_NAME='Ted Stevens Anchorage International Airport', ORIGIN_CITY='Anchorage', ORIGIN_STATE='AK', AIRPORT_DESTINATION_NAME='Seattle-Tacoma International Airport', DESTINATION_CITY='Seattle', DESTINATION_STATE='WA', DELAY_DIFF=-11)]

In [13]:
#infer schema and register whole_df as table
whole_df1.registerTempTable("data") 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#run correlation to see the relationship 
#from pyspark.ml import Statistics
#whole_df1.stat.corr("TAXI_IN","ARRIVAL_DELAY")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# QUERIES

We will build the queries of flights delay insights from 3 angles:
1. Airline
2. Airport
3. Time of the year (month and day of week)

We will use window functions to create ranking for most table results. Since our objective is about flights' delay time, ranking #1 means the delay time is highest (which is not good). The higher the rank (1 is the highest), the more delay time. There are negative values in delay time, which mean the flight departures/arrives earlier than scheduled.

## Airline

In [15]:
#find which airline have the highest numbers of flights in 2015
flights_count = spark.sql("SELECT AIRLINE, AIRLINE_NAME, COUNT(AIRLINE) num_flights FROM data GROUP BY AIRLINE,AIRLINE_NAME ORDER BY num_flights DESC")
flights_count.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-----------+
|AIRLINE|        AIRLINE_NAME|num_flights|
+-------+--------------------+-----------+
|     WN|Southwest Airline...|    1261855|
|     DL|Delta Air Lines Inc.|     875881|
|     AA|American Airlines...|     725984|
|     OO|Skywest Airlines ...|     588353|
|     EV|Atlantic Southeas...|     571977|
|     UA|United Air Lines ...|     515723|
|     MQ|American Eagle Ai...|     294632|
|     B6|     JetBlue Airways|     267048|
|     US|     US Airways Inc.|     198715|
|     AS|Alaska Airlines Inc.|     172521|
|     NK|    Spirit Air Lines|     117379|
|     F9|Frontier Airlines...|      90836|
|     HA|Hawaiian Airlines...|      76272|
|     VX|      Virgin America|      61903|
+-------+--------------------+-----------+

In [16]:
#Find the average delay difference for each airline
#positive means arrival delay is larger than departure delay
flight_delay_difference = spark.sql("SELECT AIRLINE,AIRLINE_NAME, AVG(DELAY_DIFF) delay_difference, rank(AVG(DELAY_DIFF)) OVER(ORDER BY AVG(DELAY_DIFF) DESC) as RANK_DELAY_DIFF from data GROUP BY AIRLINE,AIRLINE_NAME ORDER BY delay_difference DESC")
flight_delay_difference.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-------------------+---------------+
|AIRLINE|        AIRLINE_NAME|   delay_difference|RANK_DELAY_DIFF|
+-------+--------------------+-------------------+---------------+
|     HA|Hawaiian Airlines...| 1.5531752607146145|              1|
|     F9|Frontier Airlines...|-0.7986457986457987|              2|
|     NK|    Spirit Air Lines|-1.4113010339169916|              3|
|     OO|Skywest Airlines ...|-1.8904308841324933|              4|
|     EV|Atlantic Southeas...|-2.0302189086294415|              5|
|     US|     US Airways Inc.| -2.374790833217487|              6|
|     AS|Alaska Airlines Inc.| -2.695489357730738|              7|
|     MQ|American Eagle Ai...|-3.5093134283387912|              8|
|     VX|      Virgin America| -4.255779780564263|              9|
|     B6|     JetBlue Airways| -4.764606437136032|             10|
|     AA|American Airlines...| -5.374734022035669|             11|
|     WN|Southwest Airline...| -6.142219553558708|            

In [17]:
#save the query results to a table
flight_delay_difference.registerTempTable("rank_delay_diff")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
#Rank the airlines based on average departure delays
avg_departure_delay = spark.sql("SELECT AIRLINE, AIRLINE_NAME, AVG(DEPARTURE_DELAY) avg_dep_delay , rank(AVG(DEPARTURE_DELAY)) OVER(ORDER BY AVG(DEPARTURE_DELAY) DESC) as RANK_DEPARTURE_DELAY from data GROUP BY AIRLINE,AIRLINE_NAME ORDER BY avg_dep_delay DESC")
avg_departure_delay.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-------------------+--------------------+
|AIRLINE|        AIRLINE_NAME|      avg_dep_delay|RANK_DEPARTURE_DELAY|
+-------+--------------------+-------------------+--------------------+
|     NK|    Spirit Air Lines| 15.944765880783688|                   1|
|     UA|United Air Lines ...| 14.435441010805953|                   2|
|     F9|Frontier Airlines...| 13.350858345331709|                   3|
|     B6|     JetBlue Airways|   11.5143526744102|                   4|
|     WN|Southwest Airline...| 10.581986295158847|                   5|
|     MQ|American Eagle Ai...| 10.125188203309524|                   6|
|     VX|      Virgin America|  9.022595096521952|                   7|
|     AA|American Airlines...|  8.900856346719806|                   8|
|     EV|Atlantic Southeas...|   8.71593449776958|                   9|
|     OO|Skywest Airlines ...|  7.801103808415331|                  10|
|     DL|Delta Air Lines Inc.|   7.36925417661782|              

In [19]:
#save the query results to a table
avg_departure_delay.registerTempTable("rank_avg_dep_delay") 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
#Rank the airlines based on arrival delays
avg_departure_delay = spark.sql("SELECT AIRLINE, AIRLINE_NAME, AVG(ARRIVAL_DELAY) avg_arr_delay, rank(AVG(ARRIVAL_DELAY)) OVER(ORDER BY AVG(ARRIVAL_DELAY) DESC) as RANK_ARRIVAL_DELAY FROM data GROUP BY AIRLINE,AIRLINE_NAME ORDER BY avg_arr_delay DESC")
avg_departure_delay.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-------------------+------------------+
|AIRLINE|        AIRLINE_NAME|      avg_arr_delay|RANK_ARRIVAL_DELAY|
+-------+--------------------+-------------------+------------------+
|     NK|    Spirit Air Lines| 14.471799501705833|                 1|
|     F9|Frontier Airlines...| 12.504706404706404|                 2|
|     B6|     JetBlue Airways|  6.677860800940307|                 3|
|     EV|Atlantic Southeas...|  6.585378691739733|                 4|
|     MQ|American Eagle Ai...|  6.457873460764516|                 5|
|     OO|Skywest Airlines ...|  5.845652151300072|                 6|
|     UA|United Air Lines ...|  5.431593935741549|                 7|
|     VX|      Virgin America|  4.737705721003135|                 8|
|     WN|Southwest Airline...| 4.3749636792570525|                 9|
|     US|     US Airways Inc.| 3.7062088424131026|                10|
|     AA|American Airlines...| 3.4513721447256764|                11|
|     HA|Hawaiian Ai

In [21]:
#save the query results to a table
avg_departure_delay.registerTempTable("rank_avg_arr_delay") 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

* The table below shows the rank of average departure delay and rank of average arrival delay for each airline

In [22]:
#Merge the two tables based on airline, but only save the rankings in the new query
avg_delay_rank = spark.sql("SELECT a.AIRLINE, a.AIRLINE_NAME,a.RANK_DEPARTURE_DELAY, b.RANK_ARRIVAL_DELAY FROM rank_avg_dep_delay a JOIN rank_avg_arr_delay b on a.AIRLINE=b.AIRLINE ORDER BY a.RANK_DEPARTURE_DELAY ASC")
avg_delay_rank.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+--------------------+------------------+
|AIRLINE|        AIRLINE_NAME|RANK_DEPARTURE_DELAY|RANK_ARRIVAL_DELAY|
+-------+--------------------+--------------------+------------------+
|     NK|    Spirit Air Lines|                   1|                 1|
|     UA|United Air Lines ...|                   2|                 7|
|     F9|Frontier Airlines...|                   3|                 2|
|     B6|     JetBlue Airways|                   4|                 3|
|     WN|Southwest Airline...|                   5|                 9|
|     MQ|American Eagle Ai...|                   6|                 5|
|     VX|      Virgin America|                   7|                 8|
|     AA|American Airlines...|                   8|                11|
|     EV|Atlantic Southeas...|                   9|                 4|
|     OO|Skywest Airlines ...|                  10|                 6|
|     DL|Delta Air Lines Inc.|                  11|                13|
|     

In [23]:
#Finding the airline's average departure delay for each month
avg_dep_delay_month = spark.sql("SELECT MONTH, AIRLINE,AIRLINE_NAME, AVG(DEPARTURE_DELAY) avg_dep_delay FROM data GROUP BY MONTH,AIRLINE ,AIRLINE_NAME ORDER BY AIRLINE ASC,MONTH ASC")
avg_dep_delay_month.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------+--------------------+--------------------+
|MONTH|AIRLINE|        AIRLINE_NAME|       avg_dep_delay|
+-----+-------+--------------------+--------------------+
|    1|     AA|American Airlines...|  10.593542260208928|
|    2|     AA|American Airlines...|  10.258853469059737|
|    3|     AA|American Airlines...|    11.0866277765313|
|    4|     AA|American Airlines...|   10.55527802155764|
|    5|     AA|American Airlines...|  11.111627694972423|
|    6|     AA|American Airlines...|  14.182827822120867|
|    7|     AA|American Airlines...|   9.721148618593388|
|    8|     AA|American Airlines...|   9.357355267320468|
|    9|     AA|American Airlines...|   4.966656628979715|
|   10|     AA|American Airlines...|   5.313371455428482|
|   11|     AA|American Airlines...|   6.446925394293816|
|   12|     AA|American Airlines...|   9.118865071364143|
|    1|     AS|Alaska Airlines Inc.|  3.1782088195181086|
|    2|     AS|Alaska Airlines Inc.|  2.7038140150574996|
|    3|     AS

In [24]:
#save the query results to a table
avg_dep_delay_month.registerTempTable("average_dep_delay_tb") 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

* The table below shows the moving average of the average departure delay for each airline every two months

In [25]:
#find moving average departure delay
ma_dep_delay = spark.sql("SELECT MONTH,AIRLINE,AIRLINE_NAME, avg_dep_delay, AVG(avg_dep_delay) OVER(PARTITION BY AIRLINE ROWS BETWEEN 1 PRECEDING AND CURRENT ROW) AS moving_average_dep_delay FROM average_dep_delay_tb ORDER BY AIRLINE ASC, MONTH ASC")
ma_dep_delay.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------+--------------------+--------------------+------------------------+
|MONTH|AIRLINE|        AIRLINE_NAME|       avg_dep_delay|moving_average_dep_delay|
+-----+-------+--------------------+--------------------+------------------------+
|    1|     AA|American Airlines...|  10.593542260208928|      10.593542260208928|
|    2|     AA|American Airlines...|  10.258853469059737|      10.426197864634332|
|    3|     AA|American Airlines...|    11.0866277765313|       10.67274062279552|
|    4|     AA|American Airlines...|   10.55527802155764|      10.820952899044471|
|    5|     AA|American Airlines...|  11.111627694972423|      10.833452858265032|
|    6|     AA|American Airlines...|  14.182827822120867|      12.647227758546645|
|    7|     AA|American Airlines...|   9.721148618593388|      11.951988220357126|
|    8|     AA|American Airlines...|   9.357355267320468|       9.539251942956927|
|    9|     AA|American Airlines...|   4.966656628979715|      7.1620059481500915|
|   

In [26]:
#find average Taxi_Out for each airline
avg_taxi_out_rank = spark.sql("SELECT AIRLINE, AIRLINE_NAME, AVG(TAXI_OUT), rank(AVG(TAXI_OUT)) OVER(ORDER BY AVG(TAXI_OUT) DESC)as RANK_TAXI_OUT FROM data GROUP BY AIRLINE,AIRLINE_NAME ORDER BY RANK_TAXI_OUT ASC ")
avg_taxi_out_rank.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+------------------+-------------+
|AIRLINE|        AIRLINE_NAME|     avg(TAXI_OUT)|RANK_TAXI_OUT|
+-------+--------------------+------------------+-------------+
|     US|     US Airways Inc.| 19.00685761837739|            1|
|     OO|Skywest Airlines ...| 18.14606960978636|            2|
|     B6|     JetBlue Airways| 17.87245321216826|            3|
|     AA|American Airlines...|17.769169671960366|            4|
|     DL|Delta Air Lines Inc.|17.608081238949012|            5|
|     UA|United Air Lines ...| 17.41400330718744|            6|
|     EV|Atlantic Southeas...| 16.73697083319865|            7|
|     MQ|American Eagle Ai...|16.549322913687583|            8|
|     F9|Frontier Airlines...|15.674197766155483|            9|
|     AS|Alaska Airlines Inc.|15.093937084378146|           10|
|     VX|      Virgin America|14.755853932767359|           11|
|     NK|    Spirit Air Lines|14.604655798514512|           12|
|     WN|Southwest Airline...| 11.943682

In [27]:
#save the query results in the table
avg_taxi_out_rank.registerTempTable("rank_avg_taxi_out")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

* This table below shows the ranking of average taxi out and average departure delay for each airline

In [28]:
#join rank_arr_delay with rank_avg_taxi_out
join_out_delay_rank = spark.sql("SELECT a.AIRLINE,a.AIRLINE_NAME,a.RANK_TAXI_OUT,b.RANK_DEPARTURE_DELAY FROM rank_avg_taxi_out a LEFT OUTER JOIN rank_avg_dep_delay b on a.AIRLINE=b.AIRLINE ORDER BY RANK_TAXI_OUT ASC")
join_out_delay_rank.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-------------+--------------------+
|AIRLINE|        AIRLINE_NAME|RANK_TAXI_OUT|RANK_DEPARTURE_DELAY|
+-------+--------------------+-------------+--------------------+
|     US|     US Airways Inc.|            1|                  12|
|     OO|Skywest Airlines ...|            2|                  10|
|     B6|     JetBlue Airways|            3|                   4|
|     AA|American Airlines...|            4|                   8|
|     DL|Delta Air Lines Inc.|            5|                  11|
|     UA|United Air Lines ...|            6|                   2|
|     EV|Atlantic Southeas...|            7|                   9|
|     MQ|American Eagle Ai...|            8|                   6|
|     F9|Frontier Airlines...|            9|                   3|
|     AS|Alaska Airlines Inc.|           10|                  13|
|     VX|      Virgin America|           11|                   7|
|     NK|    Spirit Air Lines|           12|                   1|
|     WN|S

In [29]:
#save the query results in a table
join_out_delay_rank.registerTempTable("rank_dep")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
#rank the avg_taxi_in
avg_taxi_out_rank = spark.sql("SELECT AIRLINE, AIRLINE_NAME, AVG(TAXI_IN), rank(AVG(TAXI_IN)) OVER(ORDER BY AVG(TAXI_IN) DESC)as RANK_TAXI_IN FROM data GROUP BY AIRLINE,AIRLINE_NAME ORDER BY RANK_TAXI_IN ASC ")
avg_taxi_out_rank.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+------------------+------------+
|AIRLINE|        AIRLINE_NAME|      avg(TAXI_IN)|RANK_TAXI_IN|
+-------+--------------------+------------------+------------+
|     NK|    Spirit Air Lines| 9.556557398368344|           1|
|     F9|Frontier Airlines...| 9.147244295197881|           2|
|     MQ|American Eagle Ai...|  9.08409589487208|           3|
|     AA|American Airlines...| 8.876172090843598|           4|
|     UA|United Air Lines ...|  8.50809170865133|           5|
|     VX|      Virgin America| 8.186914364865967|           6|
|     EV|Atlantic Southeas...| 7.635510625870513|           7|
|     US|     US Airways Inc.| 7.631977634567749|           8|
|     DL|Delta Air Lines Inc.|7.2090576710025625|           9|
|     OO|Skywest Airlines ...| 6.933259854345498|          10|
|     HA|Hawaiian Airlines...| 6.841194327993376|          11|
|     AS|Alaska Airlines Inc.| 6.401666938471216|          12|
|     WN|Southwest Airline...| 6.167661938889348|      

In [31]:
#save the query results in a table
avg_taxi_out_rank.registerTempTable("rank_avg_taxi_in")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

* This table below shows the ranking of average taxi in and average arrival delay for each airline

In [32]:
#join the rank_avg_taxi_in with rank_avg_arr_delay
join_in_arr_rank = spark.sql("SELECT r1.AIRLINE,r1.AIRLINE_NAME,r1.RANK_TAXI_IN,r2.RANK_ARRIVAL_DELAY FROM rank_avg_taxi_in r1 LEFT OUTER JOIN rank_avg_arr_delay r2 on r1.AIRLINE=r2.AIRLINE ORDER BY RANK_TAXI_IN ASC")
join_in_arr_rank.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+------------+------------------+
|AIRLINE|        AIRLINE_NAME|RANK_TAXI_IN|RANK_ARRIVAL_DELAY|
+-------+--------------------+------------+------------------+
|     NK|    Spirit Air Lines|           1|                 1|
|     F9|Frontier Airlines...|           2|                 2|
|     MQ|American Eagle Ai...|           3|                 5|
|     AA|American Airlines...|           4|                11|
|     UA|United Air Lines ...|           5|                 7|
|     VX|      Virgin America|           6|                 8|
|     EV|Atlantic Southeas...|           7|                 4|
|     US|     US Airways Inc.|           8|                10|
|     DL|Delta Air Lines Inc.|           9|                13|
|     OO|Skywest Airlines ...|          10|                 6|
|     HA|Hawaiian Airlines...|          11|                12|
|     AS|Alaska Airlines Inc.|          12|                14|
|     WN|Southwest Airline...|          13|            

In [33]:
#save the query results in a table
join_in_arr_rank.registerTempTable("rank_arr")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

* This table below is the results of joining two tables together: rank of average departure and taxi out; rank of average arrival and taxi in

In [34]:
#Join the rank_dep table with rank_arr table, and the rank_delay_diff table
overall_taxi_delay_rank = spark.sql("SELECT a.AIRLINE, a.AIRLINE_NAME, a.RANK_TAXI_OUT,a.RANK_DEPARTURE_DELAY,b.RANK_TAXI_IN,b.RANK_ARRIVAL_DELAY,c.RANK_DELAY_DIFF FROM rank_dep a join rank_arr b on a.AIRLINE = b.AIRLINE join rank_delay_diff c on c.AIRLINE=b.AIRLINE")
overall_taxi_delay_rank.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-------------+--------------------+------------+------------------+---------------+
|AIRLINE|        AIRLINE_NAME|RANK_TAXI_OUT|RANK_DEPARTURE_DELAY|RANK_TAXI_IN|RANK_ARRIVAL_DELAY|RANK_DELAY_DIFF|
+-------+--------------------+-------------+--------------------+------------+------------------+---------------+
|     AA|American Airlines...|            4|                   8|           4|                11|             11|
|     AS|Alaska Airlines Inc.|           10|                  13|          12|                14|              7|
|     B6|     JetBlue Airways|            3|                   4|          14|                 3|             10|
|     DL|Delta Air Lines Inc.|            5|                  11|           9|                13|             13|
|     EV|Atlantic Southeas...|            7|                   9|           7|                 4|              5|
|     F9|Frontier Airlines...|            9|                   3|           2|          

In [35]:
#avg air time
avg_air = spark.sql("SELECT AIRLINE,AIRLINE_NAME,AVG(AIR_TIME) average_air FROM data GROUP BY AIRLINE,AIRLINE_NAME ORDER BY average_air DESC")

#avg distance
avg_distance = spark.sql("SELECT AIRLINE, AIRLINE_NAME, AVG(DISTANCE) average_dist FROM data GROUP BY AIRLINE,AIRLINE_NAME ORDER BY average_dist DESC")

#avg distance/air_time = average speed
avg_speed = spark.sql("SELECT AIRLINE,AIRLINE_NAME, AVG(DISTANCE)/AVG(AIR_TIME) avg_speed FROM data GROUP BY AIRLINE,AIRLINE_NAME ORDER BY avg_speed DESC")
avg_speed.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+------------------+
|AIRLINE|        AIRLINE_NAME|         avg_speed|
+-------+--------------------+------------------+
|     UA|United Air Lines ...| 7.714010082994658|
|     VX|      Virgin America| 7.698533609716087|
|     AS|Alaska Airlines Inc.| 7.586047816956597|
|     HA|Hawaiian Airlines...| 7.452663163830329|
|     AA|American Airlines...| 7.448447649136326|
|     NK|    Spirit Air Lines| 7.444524801064326|
|     F9|Frontier Airlines...|7.4263714887976855|
|     B6|     JetBlue Airways| 7.383472205010684|
|     DL|Delta Air Lines Inc.| 7.380099605413376|
|     US|     US Airways Inc.|7.2471430371648555|
|     WN|Southwest Airline...|7.1843632328746425|
|     OO|Skywest Airlines ...| 6.566282435040183|
|     EV|Atlantic Southeas...| 6.352767047909927|
|     MQ|American Eagle Ai...| 6.305131987655124|
+-------+--------------------+------------------+

* This table below shows the count of delay cases based on delay reason, grouped by each airline

In [36]:
#find reasons for delay
#airsystem
airsystem_delay= spark.sql("SELECT AIRLINE_NAME,COUNT(AIR_SYSTEM_DELAY) AS air_system, AIRLINE FROM data WHERE AIR_SYSTEM_DELAY > 1 GROUP BY AIRLINE,AIRLINE_NAME ORDER BY air_system DESC")

#weather
weather_delay= spark.sql("SELECT COUNT(WEATHER_DELAY) AS weather, AIRLINE FROM data WHERE WEATHER_DELAY > 1 GROUP BY AIRLINE ORDER BY weather DESC")

#security
security_delay= spark.sql("SELECT COUNT(SECURITY_DELAY) AS security, AIRLINE FROM data WHERE SECURITY_DELAY > 1 GROUP BY AIRLINE ORDER BY security DESC")


#airline
airline_delay= spark.sql("SELECT COUNT(AIRLINE_DELAY) AS airlines, AIRLINE FROM data WHERE AIRLINE_DELAY > 1 GROUP BY AIRLINE ORDER BY airlines DESC")


#late aircraft
late_aircraft_delay= spark.sql("SELECT COUNT(LATE_AIRCRAFT_DELAY) AS late_aircraft, AIRLINE FROM data WHERE LATE_AIRCRAFT_DELAY > 1 GROUP BY AIRLINE ORDER BY late_aircraft DESC")

airline_delay.registerTempTable("airline_del")
weather_delay.registerTempTable("weather_del")
security_delay.registerTempTable("security_del")
late_aircraft_delay.registerTempTable("late_aircraft_del")
airsystem_delay.registerTempTable("airsystem_del")

#joining
delay_reasons = spark.sql("SELECT a.AIRLINE, a.AIRLINE_NAME, a.air_system, b.weather, c.security, d.airlines, e.late_aircraft FROM airsystem_del a JOIN weather_del b ON a.AIRLINE = b.AIRLINE JOIN security_del c ON b.AIRLINE = c.AIRLINE JOIN airline_del d ON c.AIRLINE = d.AIRLINE JOIN late_aircraft_del e ON c.AIRLINE = e.AIRLINE")
delay_reasons.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------+-------+--------+--------+-------------+
|AIRLINE|        AIRLINE_NAME|air_system|weather|security|airlines|late_aircraft|
+-------+--------------------+----------+-------+--------+--------+-------------+
|     WN|Southwest Airline...|     90413|  10021|     544|  137819|       159738|
|     OO|Skywest Airlines ...|     55963|   4306|     338|   37981|        61192|
|     AA|American Airlines...|     67855|   9779|     690|   66107|        56785|
|     DL|Delta Air Lines Inc.|     61932|  11573|      58|   61036|        49268|
|     UA|United Air Lines ...|     48001|   7472|      26|   64873|        49127|
|     MQ|American Eagle Ai...|     34372|   8843|     305|   26110|        31822|
|     B6|     JetBlue Airways|     32688|   2160|     426|   37352|        31163|
|     US|     US Airways Inc.|     23021|   1644|     355|   18662|        13354|
|     NK|    Spirit Air Lines|     23919|    912|     291|   16560|        11675|
|     AS|Alaska 

## Airport

* The reason why there are null values in the tables below is because the column "ORIGINAL_AIRPORT" in flights data is type String, but there are digits in this column. After many tries, it seems to be impossible to drop the "numeric" values in the column since when imported, the data type is categorical. But we can ignore these values and focus on the non_null results

In [37]:
#find airports average departure delay
avg_airport_dep_delay = spark.sql("SELECT ORIGIN_AIRPORT, AIRPORT_ORIGIN_NAME,ORIGIN_CITY,ORIGIN_STATE, AVG(DEPARTURE_DELAY) avg_dep_delay FROM data GROUP BY ORIGIN_AIRPORT,AIRPORT_ORIGIN_NAME,ORIGIN_CITY,ORIGIN_STATE ORDER BY avg_dep_delay DESC")
avg_airport_dep_delay.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------------------+--------------------+------------+------------------+
|ORIGIN_AIRPORT| AIRPORT_ORIGIN_NAME|         ORIGIN_CITY|ORIGIN_STATE|     avg_dep_delay|
+--------------+--------------------+--------------------+------------+------------------+
|         14222|                null|                null|        null| 89.11111111111111|
|           ILG|  Wilmington Airport|          Wilmington|          DE|29.391752577319586|
|         13964|                null|                null|        null|          28.84375|
|           MVY|Martha's Vineyard...|    Marthas Vineyard|          MA| 25.90731707317073|
|           HYA|Barnstable Munici...|             Hyannis|          MA|23.182926829268293|
|         10154|                null|                null|        null|22.857142857142858|
|         10581|                null|                null|        null| 20.11111111111111|
|           STC|St. Cloud Regiona...|            St Cloud|          MN|18.692307692307693|

In [38]:
#airports avg arrival delay
avg_airport_dep_delay = spark.sql("SELECT DESTINATION_AIRPORT, AIRPORT_DESTINATION_NAME,DESTINATION_CITY,DESTINATION_STATE, AVG(ARRIVAL_DELAY) avg_arr_delay FROM data GROUP BY DESTINATION_AIRPORT,AIRPORT_DESTINATION_NAME,DESTINATION_CITY,DESTINATION_STATE ORDER BY avg_arr_delay DESC")
avg_airport_dep_delay.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------------------------+--------------------+-----------------+------------------+
|DESTINATION_AIRPORT|AIRPORT_DESTINATION_NAME|    DESTINATION_CITY|DESTINATION_STATE|     avg_arr_delay|
+-------------------+------------------------+--------------------+-----------------+------------------+
|              13964|                    null|                null|             null|          26.90625|
|                STC|    St. Cloud Regiona...|            St Cloud|               MN|23.025974025974026|
|                ILG|      Wilmington Airport|          Wilmington|               DE|21.989473684210527|
|              14025|                    null|                null|             null|              18.0|
|                TTN|    Trenton Mercer Ai...|             Trenton|               NJ| 17.43390076059399|
|                GUM|    Guam Internationa...|               Agana|               GU|  17.3202416918429|
|                OTH|    Southwest Oregon ...|         

## Time of the Year (Month and Day of Week)

In [39]:
#find avg departure delay by month
month_dep_delay = spark.sql("SELECT MONTH, AVG(DEPARTURE_DELAY), rank(AVG(DEPARTURE_DELAY)) OVER(ORDER BY AVG(DEPARTURE_DELAY) DESC)as RANK_DEPARTURE_DELAY_MONTH FROM data GROUP BY MONTH ORDER BY RANK_DEPARTURE_DELAY_MONTH ASC ")
month_dep_delay.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+--------------------------+
|MONTH|avg(DEPARTURE_DELAY)|RANK_DEPARTURE_DELAY_MONTH|
+-----+--------------------+--------------------------+
|    6|  13.986015722971977|                         1|
|    2|  11.885283967032645|                         2|
|   12|  11.783529247101587|                         3|
|    7|  11.394664197392151|                         4|
|    8|    9.93204459320268|                         5|
|    1|   9.759471188777926|                         6|
|    3|    9.66090306741507|                         7|
|    5|   9.454143844184054|                         8|
|    4|   7.721884890350968|                         9|
|   11|   6.944197242215287|                        10|
|   10|   4.982661121973602|                        11|
|    9|   4.823559775213923|                        12|
+-----+--------------------+--------------------------+

In [40]:
#save the query results to a table
month_dep_delay.registerTempTable("month_dep_delay_tb")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
#find avg departure delay by day of week 
#"1" means Monday, "7" means Sunday
day_dep_delay = spark.sql("SELECT DAY_OF_WEEK, AVG(DEPARTURE_DELAY), rank(AVG(DEPARTURE_DELAY)) OVER(ORDER BY AVG(DEPARTURE_DELAY) DESC)as RANK_DEPARTURE_DELAY_DAY FROM data GROUP BY DAY_OF_WEEK ORDER BY RANK_DEPARTURE_DELAY_DAY ASC ")
day_dep_delay.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+------------------------+
|DAY_OF_WEEK|avg(DEPARTURE_DELAY)|RANK_DEPARTURE_DELAY_DAY|
+-----------+--------------------+------------------------+
|          1|  10.871775063646947|                       1|
|          4|   9.956913497060805|                       2|
|          5|   9.429767977851792|                       3|
|          7|   9.402855627605891|                       4|
|          2|   9.164522265904088|                       5|
|          3|   8.645276690756555|                       6|
|          6|   7.827415699173568|                       7|
+-----------+--------------------+------------------------+

In [42]:
#save query results to a table
day_dep_delay.registerTempTable("day_dep_delay_tb")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
#find avg arrical delay by month
month_arr_delay = spark.sql("SELECT MONTH, AVG(ARRIVAL_DELAY), rank(AVG(ARRIVAL_DELAY)) OVER(ORDER BY AVG(ARRIVAL_DELAY) DESC)as RANK_ARRIVAL_DELAY_MONTH FROM data GROUP BY MONTH ORDER BY RANK_ARRIVAL_DELAY_MONTH ASC ")
month_arr_delay.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------------+------------------------+
|MONTH| avg(ARRIVAL_DELAY)|RANK_ARRIVAL_DELAY_MONTH|
+-----+-------------------+------------------------+
|    6|  9.601590351569554|                       1|
|    2|  8.320500020850556|                       2|
|    7| 6.4317747052785466|                       3|
|   12|  6.092902747824754|                       4|
|    1|  5.813582983416227|                       5|
|    3|   4.92067265685641|                       6|
|    8|  4.607372469025074|                       7|
|    5|  4.485018615679651|                       8|
|    4| 3.1631900611579318|                       9|
|   11|  1.100783576682592|                      10|
|    9|-0.7725709883956179|                      11|
|   10| -0.780389663641748|                      12|
+-----+-------------------+------------------------+

In [44]:
#save query results to a table
month_arr_delay.registerTempTable("month_arr_delay_tb")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
#find avg arrical delay by day-of-week
day_arr_delay = spark.sql("SELECT DAY_OF_WEEK, AVG(ARRIVAL_DELAY), rank(AVG(ARRIVAL_DELAY)) OVER(ORDER BY AVG(ARRIVAL_DELAY) DESC)as RANK_ARRIVAL_DELAY_DAY FROM data GROUP BY DAY_OF_WEEK ORDER BY RANK_ARRIVAL_DELAY_DAY ASC ")
day_arr_delay.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------------+----------------------+
|DAY_OF_WEEK|avg(ARRIVAL_DELAY)|RANK_ARRIVAL_DELAY_DAY|
+-----------+------------------+----------------------+
|          1| 6.002063450202781|                     1|
|          4| 5.668333554807981|                     2|
|          5| 4.764079085069422|                     3|
|          2| 4.246040906503392|                     4|
|          7|3.9582932010890217|                     5|
|          3|3.8454678490872136|                     6|
|          6|1.8528920108155913|                     7|
+-----------+------------------+----------------------+

In [46]:
#save query results to a table
day_arr_delay.registerTempTable("day_arr_delay_tb")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

* This table below shows the rank of average departure delay and rank of average arrival delay by month

In [47]:
#Join tables: 
#departure and arrival delay for each month
#departure and arrival delay for day of week
month_delay = spark.sql("SELECT a.MONTH,a.RANK_DEPARTURE_DELAY_MONTH,b.RANK_ARRIVAL_DELAY_MONTH from month_dep_delay_tb a join month_arr_delay_tb b on a.MONTH=b.MONTH ORDER BY MONTH ASC")
month_delay.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------------+------------------------+
|MONTH|RANK_DEPARTURE_DELAY_MONTH|RANK_ARRIVAL_DELAY_MONTH|
+-----+--------------------------+------------------------+
|    1|                         6|                       5|
|    2|                         2|                       2|
|    3|                         7|                       6|
|    4|                         9|                       9|
|    5|                         8|                       8|
|    6|                         1|                       1|
|    7|                         4|                       3|
|    8|                         5|                       7|
|    9|                        12|                      11|
|   10|                        11|                      12|
|   11|                        10|                      10|
|   12|                         3|                       4|
+-----+--------------------------+------------------------+

* This table below shows the rank of average departure delay and rank of average arrival delay by day of week

In [48]:
day_delay = spark.sql("SELECT a.DAY_OF_WEEK, a.RANK_DEPARTURE_DELAY_DAY,b.RANK_ARRIVAL_DELAY_DAY from day_dep_delay_tb a join day_arr_delay_tb b on a.DAY_OF_WEEK=b.DAY_OF_WEEK ORDER BY DAY_OF_WEEK ASC")
day_delay.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------------------+----------------------+
|DAY_OF_WEEK|RANK_DEPARTURE_DELAY_DAY|RANK_ARRIVAL_DELAY_DAY|
+-----------+------------------------+----------------------+
|          1|                       1|                     1|
|          2|                       5|                     4|
|          3|                       6|                     6|
|          4|                       2|                     2|
|          5|                       3|                     3|
|          6|                       7|                     7|
|          7|                       4|                     5|
+-----------+------------------------+----------------------+